# Inits

In [1]:
import pandas as pd
from sklearn import svm

# Reading and formatting data

In [2]:
df = pd.read_excel("nar-00915-met-h-2016-File011.xlsx", skiprows = 1)
df = df.rename(columns = {"sgRNA_id": "ID",
                          "sgRNA_sequence": "sequence",
                          "target_gene": "gene",
                          "distance_to_CAGE": "distance",
                          "sgRNA_SSC_score": "score"})
df = df.loc[:, ["ID", "sequence", "gene", "distance", "score"]]
print(df)

               ID                sequence   gene  distance     score
0     id_PRDM2-10  GTTTTACAGTACATCTGAAAGT  PRDM2    -44483       NaN
1      id_PRDM2-5   GCTGACCTGCCTTCTAACAAG  PRDM2    -44396  0.177674
2      id_PRDM2-3   GACAGTATCACAGATCAAGAA  PRDM2    -44300  0.230363
3    id_PRDM2-SD1    GAGCCAGCGCCCGGAGCAAG  PRDM2        26  0.089286
4    id_PRDM2-SD3    GCTCCGCTTGCTCCGGGCGC  PRDM2        32 -0.152766
..            ...                     ...    ...       ...       ...
150  id_KDM5C-SD2   GGGAGGTAACTGTGGTGGCGA  KDM5C        54  0.003474
151  id_KDM5C-SD1    GCGCTGAAGGCCGCGGCGTT  KDM5C        33 -0.045968
152    id_KDM5C-1    GGCTGGTATGGGTTTCTATT  KDM5C      -134 -0.028234
153    id_KDM5D-3    GACGATTGGTAAGCAGCTAC  KDM5D        54  0.044147
154    id_KDM5D-6   GTACCACACTGCAGGGCCCGG  KDM5D        -1  0.029813

[155 rows x 5 columns]


c:\Users\jacks\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# testing test